# Manual Tracing of AI Code

This notebook demonstrates how to trace AI code execution using Maxim's manual tracing approach. We'll analyze the code step by step to understand the flow and behavior of the AI system.

In [15]:
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

True

## Initialize Maxim logger

In [16]:
from maxim import Maxim

# This automatically creates logger by reading MAXIM_API_KEY and MAXIM_LOG_REPO_ID from environment variables
logger = Maxim({}).logger({})

[MaximSDK] Initializing Maxim AI(v3.6.2)


## Creating a trace

In [17]:
trace = logger.trace({"name": "test-trace"})
trace.set_input("Hi!")
trace.set_output("Hello, world!")

[MaximSDK] Id is required. Entity: TRACE. Config: {'name': 'test-trace'}. We are generating a random id for you.


## Adding an LLM call

In [18]:
import os
from openai import OpenAI

client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "Write a haiku about recursion in programming."},
]

try:
    # Create a generation
    generation = trace.generation({
        "model": "gpt-4o-mini",
        "messages": messages,
    })
    # Create a chat completion request
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages,        
    )
    # Add result to the generation
    generation.result(response)
    # Extract response text and usage
    response_text = response.choices[0].message.content
    usage = response.usage    
    print(response_text,usage)    
except Exception as e:
    # Add error to the generation
    generation.error(e)
finally:
    generation.end()

[MaximSDK] Id is required. Entity: GENERATION. Config: {'model': 'gpt-4o-mini', 'messages': [{'role': 'system', 'content': 'You are a helpful assistant.'}, {'role': 'user', 'content': 'Write a haiku about recursion in programming.'}]}. We are generating a random id for you.


Code calls itself deep,  
Functions nested, layers stack,  
Endless loops embrace. CompletionUsage(completion_tokens=18, prompt_tokens=26, total_tokens=44, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0))


## Adding a retrieval to the trace

In [19]:
import uuid
retrieval = trace.retrieval({
    "id": str(uuid.uuid4()),
})
retrieval.input("What is the capital of France?")
retrieval.output(['{\"embedding_model_name\": \"text-embedding-ada-002\"}'])